In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import psycopg2

db_params = {
    'host': 'localhost',
    'port': '5432',
    'user': 'postgres',
    'password': 'AAAAAA',
    'database': 'BiblioSphere'
}

dsn = f"host={db_params['host']} port={db_params['port']} user={db_params['user']} password={db_params['password']} dbname={db_params['database']}"

conn = psycopg2.connect(dsn)

# Widgets
output_resultado = widgets.Output()

# Função para realizar a inclusão do livro
def confirmar_inclusao(b):
    output_resultado.clear_output()
    titulo = titulo_inclusao_widget.value
    id_editora = id_editora_inclusao_widget.value
    isbn = isbn_inclusao_widget.value
    edicao = edicao_inclusao_widget.value
    data_publicacao = data_publicacao_inclusao_widget.value
    quantidade_exemplares = quantidade_exemplares_inclusao_widget.value
    paginas = paginas_inclusao_widget.value

    # Verifica se a data de publicação é None e, se sim, define como uma string vazia
    data_publicacao = '' if data_publicacao is None else str(data_publicacao)

    # Insira a tupla do livro no banco de dados
    query_inclusao = f"INSERT INTO livro (Paginas, ISBN, Titulo, Edição, Data_publicação, Quantidade_exemplares, ID_editora) VALUES ({paginas}, {isbn}, '{titulo}', '{edicao}', '{data_publicacao}', {quantidade_exemplares}, {id_editora});"
    
    with conn.cursor() as cursor:
        cursor.execute(query_inclusao)
        conn.commit()
    
    with output_resultado:
        print(f"Livro '{titulo}' incluído com sucesso!")

# Função para confirmar a inclusão de pessoa
def confirmar_inclusao_pessoa(b):
    output_resultado.clear_output()

    # Obtenha os valores dos widgets
    nome_pessoa = nome_pessoa_widget.value
    id_pessoa = id_pessoa_widget.value
    foto_pessoa = foto_pessoa_widget.value

    # Verifique se a foto da pessoa é None e, se sim, defina como uma string vazia
    foto_pessoa = '' if foto_pessoa is None else str(foto_pessoa)

    # Insira a tupla da pessoa no banco de dados
    query_inclusao_pessoa = f"INSERT INTO pessoa (nome, id_pessoa, foto) VALUES ('{nome_pessoa}', {id_pessoa}, '{foto_pessoa}');"

    try:
        with conn.cursor() as cursor:
            cursor.execute(query_inclusao_pessoa)
            conn.commit()

        with output_resultado:
            print(f"Pessoa '{nome_pessoa}' incluída com sucesso!")
    except psycopg2.Error as e:
        with output_resultado:
            print(f"Erro ao incluir pessoa: {str(e)}")


# Widgets para inclusão de pessoa
nome_pessoa_widget = widgets.Text(value='', description='Nome da Pessoa:')
id_pessoa_widget = widgets.IntText(value=0, description='ID da Pessoa:')
foto_pessoa_widget = widgets.Text(value='', description='URL da Foto da Pessoa:')

# Botão para confirmar a inclusão da pessoa
botao_confirmar_inclusao_pessoa = widgets.Button(description="Incluir Pessoa")

# Associe a função de inclusão ao botão de inclusão
botao_confirmar_inclusao_pessoa.on_click(confirmar_inclusao_pessoa)

# Associe a função ao botão de inclusão de pessoa
botao_confirmar_inclusao_pessoa.on_click(confirmar_inclusao_pessoa)


# Função para incluir um membro
def incluir_membro():
    output_resultado.clear_output()

    # Obtenção dos valores dos widgets
    cpf_membro = cpf_membro_widget.value
    data_filiacao = data_filiacao_widget.value
    historico_emprestimo = historico_emprestimo_widget.value

    # Inserção da tupla do membro no banco de dados
    query_inclusao_membro = f"INSERT INTO Membro (CPF, Data_filiação, Historico_emprestimo) VALUES ('{cpf_membro}', '{data_filiacao}', '{historico_emprestimo}') RETURNING ID_pessoa;"
    
    try:
        # Execução da query para incluir o membro
        with conn.cursor() as cursor:
            cursor.execute(query_inclusao_membro)
            id_membro = cursor.fetchone()[0]

            conn.commit()

        with output_resultado:
            print(f"Membro com CPF '{cpf_membro}' incluído com sucesso!")
    except psycopg2.Error as e:
        conn.rollback()
        with output_resultado:
            print(f"Erro ao incluir membro: {str(e)}")


# Função para incluir uma editora
def incluir_editora(b):
    output_resultado.clear_output()

    # Obtenção dos valores dos widgets
    nome_editora = nome_editora_widget.value
    rua_editora = rua_editora_widget.value
    bairro_editora = bairro_editora_widget.value
    cidade_editora = cidade_editora_widget.value
    contato_editora = contato_editora_widget.value

    # Inserção da tupla da editora no banco de dados
    query_inclusao_editora = f"INSERT INTO Editora (Nome, Bairro, Rua, Cidade) VALUES ('{nome_editora}', '{bairro_editora}', '{rua_editora}', '{cidade_editora}') RETURNING ID_editora;"
    
    try:
        # Execução da query para incluir a editora
        with conn.cursor() as cursor:
            cursor.execute(query_inclusao_editora)
            id_editora = cursor.fetchone()[0]
            
            # Inserção do contato na tabela Editora_contato
            query_inclusao_contato = f"INSERT INTO Editora_contato (Contato, ID_editora) VALUES ({contato_editora}, {id_editora});"
            cursor.execute(query_inclusao_contato)
            
            conn.commit()

        with output_resultado:
            print(f"Editora '{nome_editora}' incluída com sucesso!")
    except psycopg2.Error as e:
        conn.rollback()
        with output_resultado:
            print(f"Erro ao incluir editora: {str(e)}")
 
# Widgets
output_resultado = widgets.Output()

# Função para criar a aba de inclusão
def criar_aba_inclusao(b):
    output_resultado.clear_output()

    # Botão para acionar os widgets de inclusão de livro
    botao_incluir_livro = widgets.Button(description="Incluir Livro")
    
    # Botão para acionar os widgets de inclusão de pessoa
    botao_incluir_pessoa = widgets.Button(description="Incluir Pessoa")

    # Botão para acionar os widgets de inclusão de membro
    botao_incluir_membro = widgets.Button(description="Incluir Membro")

    # Botão para acionar os widgets de inclusão de editora
    botao_incluir_editora = widgets.Button(description="Incluir Editora")

    
    # Função para exibir os widgets de inclusão de livro
    def exibir_widgets_inclusao_livro(b):
        output_resultado.clear_output()

        # Widgets para a inclusão de livro
        global titulo_inclusao_widget, id_editora_inclusao_widget, isbn_inclusao_widget, edicao_inclusao_widget, data_publicacao_inclusao_widget, quantidade_exemplares_inclusao_widget, paginas_inclusao_widget
        titulo_inclusao_widget = widgets.Text(value='', description='Título do Livro:')
        id_editora_inclusao_widget = widgets.IntText(value=0, description='ID da Editora:')
        isbn_inclusao_widget = widgets.IntText(value=0, description='ISBN do Livro:')
        edicao_inclusao_widget = widgets.Text(value='', description='Edição do Livro:')
        data_publicacao_inclusao_widget = widgets.DatePicker(description='Data de Publicação:')
        quantidade_exemplares_inclusao_widget = widgets.IntText(value=0, description='Quantidade de Exemplares:')
        paginas_inclusao_widget = widgets.IntText(value=0, description='Número de Páginas do Livro')

        # Botão para confirmar a inclusão do livro
        botao_confirmar_inclusao = widgets.Button(description="Incluir Livro")

        # Associe a função de inclusão ao botão de inclusão
        botao_confirmar_inclusao.on_click(confirmar_inclusao)

        # Exiba os widgets para inclusão do livro
        with output_resultado:
            display(widgets.VBox([titulo_inclusao_widget, id_editora_inclusao_widget, isbn_inclusao_widget, edicao_inclusao_widget, data_publicacao_inclusao_widget, quantidade_exemplares_inclusao_widget, paginas_inclusao_widget, botao_confirmar_inclusao]))

    
    # Função para exibir os widgets de inclusão de pessoa
    def exibir_widgets_inclusao_pessoa(b):
        output_resultado.clear_output()

        # Widgets para a inclusão de pessoa
        global nome_pessoa_widget, id_pessoa_widget, foto_pessoa_widget
        nome_pessoa_widget = widgets.Text(value='', description='Nome da Pessoa:')
        id_pessoa_widget = widgets.IntText(value=0, description='ID da Pessoa:')
        foto_pessoa_widget = widgets.Text(value='', description='URL da Foto da Pessoa:')

        # Botão para confirmar a inclusão da pessoa
        botao_confirmar_inclusao_pessoa = widgets.Button(description="Incluir Pessoa")

        # Associe a função de inclusão ao botão de inclusão
        botao_confirmar_inclusao_pessoa.on_click(confirmar_inclusao_pessoa)

        # Exiba os widgets para inclusão de pessoa
        with output_resultado:
            display(widgets.VBox([nome_pessoa_widget, id_pessoa_widget, foto_pessoa_widget, botao_confirmar_inclusao_pessoa]))
    

    # Função para exibir os widgets de inclusão de membro
    def exibir_widgets_inclusao_membro(b):
        output_resultado.clear_output()

        global cpf_membro_widget, data_filiacao_widget, historico_emprestimo_widget
    
        # Definição dos widgets para inclusão de membro
        cpf_membro_widget = widgets.Text(value='', description='CPF do Membro:')
        data_filiacao_widget = widgets.DatePicker(description='Data de Filiação:')
        historico_emprestimo_widget = widgets.Textarea(value='', description='Histórico de Empréstimo:')
    
        # Botão para confirmar a inclusão do membro
        botao_confirmar_inclusao_membro = widgets.Button(description="Incluir Membro")
        # Associando à função de inclusão
        botao_confirmar_inclusao_membro.on_click(lambda b: incluir_membro())
  

        # Exibição dos widgets para inclusão de membro
        with output_resultado:
            display(widgets.VBox([cpf_membro_widget, data_filiacao_widget, historico_emprestimo_widget, botao_confirmar_inclusao_membro]))

    
    # Função para exibir os widgets de inclusão de editora
    def exibir_widgets_inclusao_editora(b):
        output_resultado.clear_output()

        global nome_editora_widget, rua_editora_widget, bairro_editora_widget, cidade_editora_widget, contato_editora_widget
        
        # Definição dos widgets para inclusão de editora
        nome_editora_widget = widgets.Text(value='', description='Nome da Editora:')
        rua_editora_widget = widgets.Text(value='', description='Rua:')
        bairro_editora_widget = widgets.Text(value='', description='Bairro:')
        cidade_editora_widget = widgets.Text(value='', description='Cidade:')
        contato_editora_widget = widgets.IntText(value=0, description='Contato:')
        
        # Botão para confirmar a inclusão da editora
        botao_confirmar_inclusao_editora = widgets.Button(description="Incluir Editora")
        botao_confirmar_inclusao_editora.on_click(incluir_editora)  # Associando à função de inclusão

        # Exibição dos widgets para inclusão de editora
        with output_resultado:
            display(widgets.VBox([nome_editora_widget, rua_editora_widget, bairro_editora_widget, cidade_editora_widget, contato_editora_widget, botao_confirmar_inclusao_editora]))
    

    # Associe a função de exibição ao botão "Incluir Livro"
    botao_incluir_livro.on_click(exibir_widgets_inclusao_livro)
    
    # Associe a função de exibição ao botão "Incluir Pessoa"
    botao_incluir_pessoa.on_click(exibir_widgets_inclusao_pessoa)

    # Associe a função de exibição ao botão "Incluir Membro"
    botao_incluir_membro.on_click(exibir_widgets_inclusao_membro)

    # Associe a função de exibição ao botão "Incluir Editora"
    botao_incluir_editora.on_click(exibir_widgets_inclusao_editora)

    # Exiba os botões "Incluir Livro" e "Incluir Pessoa"
    with output_resultado:
        display(widgets.HBox([botao_incluir_livro, botao_incluir_pessoa, botao_incluir_editora, botao_incluir_membro]))

# Associe a função ao botão de inclusão
botao_inclusao = widgets.Button(description="Inclusão") 
botao_inclusao.on_click(criar_aba_inclusao)

# Função para excluir um livro com base no ISBN
def confirmar_exclusao_livro(isbn):
    output_resultado.clear_output()
    
    # Verifica se o livro com o ISBN fornecido existe antes de excluir
    query_verificacao = f"SELECT * FROM livro WHERE isbn = '{isbn}';"
    df_verificacao = pd.read_sql(query_verificacao, conn)
    
    if not df_verificacao.empty:
        # O livro existe, pode ser excluído
        query_exclusao = f"DELETE FROM livro WHERE isbn = '{isbn}';"
        
        with conn.cursor() as cursor:
            cursor.execute(query_exclusao)
            conn.commit()

        with output_resultado:
            print(f"Livro com ISBN '{isbn}' excluído com sucesso!")
    else:
        # O livro não existe
        with output_resultado:
            print(f"Livro com ISBN '{isbn}' não encontrado. Por favor, forneça um ISBN válido para a exclusão.")

# Função para excluir um autor com base no ID
def confirmar_exclusao_autor(id_autor):
    output_resultado.clear_output()

    # Verifica se o autor com o ID fornecido existe antes de excluir
    query_verificacao = f"SELECT * FROM autor WHERE id_autor = {id_autor};"
    df_verificacao = pd.read_sql(query_verificacao, conn)

    if not df_verificacao.empty:
        # Exclua as entradas relacionadas na tabela "escreve"
        query_exclusao_escreve = f"DELETE FROM escreve WHERE id_autor = {id_autor};"
        with conn.cursor() as cursor:
            cursor.execute(query_exclusao_escreve)
            conn.commit()

        # O autor existe, pode ser excluído
        query_exclusao_autor = f"DELETE FROM autor WHERE id_autor = {id_autor};"
        with conn.cursor() as cursor:
            cursor.execute(query_exclusao_autor)
            conn.commit()

        with output_resultado:
            print(f'Autor com ID {id_autor} excluído com sucesso!')
    else:
        # O autor não existe
        with output_resultado:
            print(f'Autor com ID {id_autor} não encontrado. Por favor, forneça um ID válido para a exclusão.')

# Função para excluir um membro com base no CPF
def confirmar_exclusao_membro(cpf_membro):
    output_resultado.clear_output()
    
    try:
        # Verifica se o membro com o CPF fornecido existe antes de excluir
        query_verificacao = f"SELECT * FROM Membro WHERE CPF = '{cpf_membro}';"
        with conn.cursor() as cursor:
            cursor.execute(query_verificacao)
            membro_info = cursor.fetchone()

            if membro_info:
                # Exclui o membro da tabela Membro
                query_exclusao_membro = f"DELETE FROM Membro WHERE CPF = '{cpf_membro}';"
                cursor.execute(query_exclusao_membro)
                conn.commit()
                
                with output_resultado:
                    print(f"Membro com CPF {cpf_membro} excluído com sucesso!")
            else:
                # O membro não existe
                print(f"Membro com CPF {cpf_membro} não encontrado. Forneça um CPF válido para a exclusão.")
    except psycopg2.Error as e:
        with output_resultado:
            print(f"Erro ao excluir membro: {str(e)}")


# Função para excluir uma editora com base no ID
def confirmar_exclusao_editora(id_editora):
    output_resultado.clear_output()
    
    try:
        # Verifica se a editora com o ID fornecido existe antes de excluir
        query_verificacao = f"SELECT * FROM Editora WHERE ID_editora = {id_editora};"
        with conn.cursor() as cursor:
            cursor.execute(query_verificacao)
            editora_info = cursor.fetchone()

            if editora_info:
                # A editora existe, exclui as entradas relacionadas na tabela Editora_contato
                query_exclusao_contato = f"DELETE FROM Editora_contato WHERE ID_editora = {id_editora};"
                cursor.execute(query_exclusao_contato)
                
                # Exclui a editora da tabela Editora
                query_exclusao_editora = f"DELETE FROM Editora WHERE ID_editora = {id_editora};"
                cursor.execute(query_exclusao_editora)
                conn.commit()
                
                with output_resultado:
                    print(f"Editora com ID {id_editora} excluída com sucesso!")
            else:
                # A editora não existe
                print(f"Editora com ID {id_editora} não encontrada. Forneça um ID válido para a exclusão.")
    except psycopg2.Error as e:
        with output_resultado:
            print(f"Erro ao excluir editora: {str(e)}")

# EXCLUSÃO
def criar_aba_exclusao(b):
    output_resultado.clear_output()

    # Crie um widget de texto para o ISBN de exclusão de livro
    isbn_exclusao_widget = widgets.Text(value='', description='ISBN do Livro:')
    botao_confirmar_exclusao_livro = widgets.Button(description="Confirmar Exclusão de Livro")
    botao_confirmar_exclusao_livro.on_click(lambda b: confirmar_exclusao_livro(isbn_exclusao_widget.value))

    # Crie um widget de texto para o ID de exclusão de autor
    id_autor_exclusao_widget = widgets.IntText(value=0, description='ID do Autor:')
    botao_confirmar_exclusao_autor = widgets.Button(description="Confirmar Exclusão de Autor")
    botao_confirmar_exclusao_autor.on_click(lambda b: confirmar_exclusao_autor(id_autor_exclusao_widget.value))

    # Adiciona um widget de texto para o CPF de exclusão de membro
    cpf_exclusao_widget = widgets.Text(value='', description='CPF do Membro:')
    botao_confirmar_exclusao_membro = widgets.Button(description="Confirmar Exclusão de Membro")
    botao_confirmar_exclusao_membro.on_click(lambda b: confirmar_exclusao_membro(cpf_exclusao_widget.value))

    # Crie um widget de texto para o ID de exclusão de editora
    id_editora_exclusao_widget = widgets.IntText(value=0, description='ID da Editora:')
    botao_confirmar_exclusao_editora = widgets.Button(description="Confirmar Exclusão de Editora")
    botao_confirmar_exclusao_editora.on_click(lambda b: confirmar_exclusao_editora(id_editora_exclusao_widget.value))

    
    with output_resultado:
        display(widgets.HBox([isbn_exclusao_widget, botao_confirmar_exclusao_livro]))
        display(widgets.HBox([id_autor_exclusao_widget, botao_confirmar_exclusao_autor]))
        display(widgets.HBox([id_editora_exclusao_widget, botao_confirmar_exclusao_editora]))
        display(widgets.HBox([cpf_exclusao_widget, botao_confirmar_exclusao_membro]))


# Associe a função ao botão de exclusão
botao_exclusao = widgets.Button(description="Exclusão")
botao_exclusao.on_click(criar_aba_exclusao)

# REALIZA AS CONSULTAS
def executar_consulta(b):
    output_resultado.clear_output()
    
    # Exiba botões após a consulta inicial
    botao_consulta_autor = widgets.Button(description="Consultar")
    botao_consulta_livro = widgets.Button(description="Consultar")
    botao_consultar_membros = widgets.Button(description="Consultar")
    botao_consulta_editora = widgets.Button(description="Consultar") 
    botao_consulta_pessoa = widgets.Button(description="Consultar")
    botao_consultar_penalidade = widgets.Button(description="Consultar")
    botao_consultar_exemplares = widgets.Button(description="Consultar")
    botao_consultar_membros_bloqueados = widgets.Button(description="Consultar")
    botao_calcular_media_dias_atraso = widgets.Button(description="Consultar")
    botao_calcular_total_multas_por_mes = widgets.Button(description="Consultar")
    botao_generos_mais_populares = widgets.Button(description="Consultar")
    botao_media_dias_atraso_autores_britanicos = widgets.Button(description="Consultar")
    botao_quantidade_exemplares_usados_ficcao_cientifica = widgets.Button(description="Consultar")
    botao_media_multas_ficcao_cientifica_ultimos_6_meses = widgets.Button(description="Consultar")
    botao_membros_que_tiveram_mais_atraso = widgets.Button(description="Consultar")


    # Widgets
    nacionalidade_autor_widget = widgets.Text(value='', description='Nacionalidade do autor:')
    
    label_consulta_autor = widgets.HTML(value="<b>Consultar autor por nacionalidade:</b>", layout=widgets.Layout(width='250px'))
    
    titulo_livro_widget = widgets.Text(value='', description='Título do livro:')
    
    label_consulta_livro = widgets.HTML(value="<b>Consultar livro por título:</b>", layout=widgets.Layout(width='250px'))
    
    cpf_membro_widget = widgets.IntText(value=0, description='CPF do membro:')
    
    label_consulta_membro = widgets.HTML(value="<b>Consultar membros:</b>", layout=widgets.Layout(width='250px'))
    
    label_consulta_pessoa = widgets.HTML(value="<b>Consultar pessoa por nome:</b>", layout=widgets.Layout(width='250px'))

    nome_editora_consulta_widget = widgets.Text(value='', description='Nome da editora:')
    
    label_consulta_editora = widgets.HTML(value="<b>Consultar editora por nome:</b>", layout=widgets.Layout(width='250px'))

    label_consulta_penalidade = widgets.HTML(value="<b>Consultar total de penalidades e o valor médio de multas:</b>", layout=widgets.Layout(width='250px'))

    label_consulta_exemplares = widgets.HTML(value="<b>Consultar livro com mais mais exemplares:</b>", layout=widgets.Layout(width='250px'))

    label_consulta_bloqueados = widgets.HTML(value="<b>Consultar membros bloqueados:</b>", layout=widgets.Layout(width='250px'))

    label_calcular_media_dias_atraso= widgets.HTML(value="<b>Consultar media de dias de atraso:</b>", layout=widgets.Layout(width='250px'))

    label_calcular_total_multas_por_mes = widgets.HTML(value="<b>Consultar total de multas por mês de 2023:</b>", layout=widgets.Layout(width='250px'))

    label_generos_mais_populares = widgets.HTML(value="<b>Consultar generos mais populares:</b>", layout=widgets.Layout(width='250px'))
    
    label_media_dias_atraso_autores_britanicos = widgets.HTML(value="<b>Consultar media de dias de atraso dos autores britnicos:</b>", layout=widgets.Layout(width='250px'))

    label_quantidade_exemplares_usados_ficcao_cientifica = widgets.HTML(value="<b>Consultar quantidade de exemplares de ficcao cientifica:</b>", layout=widgets.Layout(width='250px'))
    
    label_media_multas_ficcao_cientifica_ultimos_6_meses = widgets.HTML(value="<b>Consultar média de multas dos livros do gênero Ficção Científica nos últimos 6 meses:</b>", layout=widgets.Layout(width='250px'))

    label_membros_com_mais_atrasos = widgets.HTML(value="<b>Consultar membros que tiveram mais atrasos:</b>", layout=widgets.Layout(width='250px'))

    # Função para consultar pessoa por nome
    def consultar_pessoa_por_nome(b):
        output_resultado.clear_output()
        nome_pessoa = nome_pessoa_widget.value
        query = f"SELECT * FROM pessoa WHERE nome ILIKE '%{nome_pessoa}%';"
        df = pd.read_sql(query, conn)
        with output_resultado:
            display(df)


    # Função para consultar autor por nacionalidade
    def consultar_autor_por_nacionalidade(b):
        output_resultado.clear_output()
        nacionalidade_autor = nacionalidade_autor_widget.value
        query = f"SELECT * FROM autor WHERE nacionalidade ILIKE '%{nacionalidade_autor}%';"
        df = pd.read_sql(query, conn)
        with output_resultado:
            display(df)
    
    # Função para consultar livro por título
    def consultar_livro_por_titulo(b):
        output_resultado.clear_output()
        titulo_livro = titulo_livro_widget.value
        query = f"SELECT * FROM livro WHERE titulo ILIKE '%{titulo_livro}%';"
        df = pd.read_sql(query, conn)
        with output_resultado:
            display(df)
    
    # Função para consultar membro por cpf
    def consultar_membros_por_cpf(b):
        output_resultado.clear_output()
        cpf_membro = cpf_membro_widget.value
        query = f"SELECT * FROM membro WHERE cpf = {cpf_membro};"
        df = pd.read_sql(query, conn)
        with output_resultado:
            display(df)


     # Função para consultar editora por nome
    def consultar_editora(b):
        output_resultado.clear_output()
        nome_editora = nome_editora_consulta_widget.value
        query = f"SELECT * FROM Editora WHERE Nome ILIKE '%{nome_editora}%';"
        try:
            with conn.cursor() as cursor:
                cursor.execute(query)
                editoras_info = cursor.fetchall()
                if editoras_info:
                    with output_resultado:
                        print(f"Editoras com o nome '{nome_editora}':")
                        for editora_info in editoras_info:
                            print(f"ID: {editora_info[0]}, Nome: {editora_info[1]}, Bairro: {editora_info[2]}, Rua: {editora_info[3]}, Cidade: {editora_info[4]}")
                else:
                    with output_resultado:
                        print(f"Nenhuma editora encontrada com o nome '{nome_editora}'.")
        except psycopg2.Error as e:
            with output_resultado:
                print(f"Erro ao consultar editora: {str(e)}")
                
    
    # Função par consultar penalidades
    def consultar_penalidade(b):
        output_resultado.clear_output()
        
        query = '''
        SELECT COUNT(Penalidade.ID_penalidade) AS Total_Penalidades, AVG(Penalidade.Valor_multa) AS Valor_Multa_Medio
        FROM Penalidade;
        '''
        
        df = pd.read_sql(query, conn)
        
        with output_resultado:
            display(df)    


    # Função para consultar o livro com mais exemplares
    def consultar_livro_com_mais_exemplares(b):
        output_resultado.clear_output() 
    
        query = '''
        SELECT Livro.Titulo, COUNT(Exemplar.ID_exemplar) AS Quantidade_Exemplares
        FROM Livro
        JOIN Exemplar ON Livro.ISBN = Exemplar.ISBN
        GROUP BY Livro.Titulo
        ORDER BY Quantidade_Exemplares DESC
        LIMIT 1;
        '''
        
        df = pd.read_sql(query, conn)
        
        with output_resultado:
            display(df)

    # Função para consultar membros bloqueados com o nome da pessoa
    def consultar_membros_bloqueados(b):
        output_resultado.clear_output()
        
        query = '''
        SELECT Pessoa.nome, COUNT(Membro.ID_pessoa) AS Membros_Bloqueados
        FROM Pessoa
        JOIN Membro ON Pessoa.ID_pessoa = Membro.ID_pessoa
        JOIN Penalidade ON Membro.ID_pessoa = Penalidade.ID_pessoa
        WHERE Penalidade.Bloqueio_emprestimo = 'Sim'
        GROUP BY Pessoa.nome;
        '''
        
        df = pd.read_sql(query, conn)
        
        with output_resultado:
            display(df)

    
    def calcular_media_dias_atraso(b):
        output_resultado.clear_output()
        
        query = '''
        SELECT ROUND(AVG(Emprestimo.Dias_de_atraso)::NUMERIC, 2) AS Media_Dias_Atraso
        FROM Emprestimo;
        '''
        
        df = pd.read_sql(query, conn)
        
        with output_resultado:
            display(df)

    # Função para calcular o total de multas aplicadas em cada mês do ano de 2023 com o nome do mês
    def calcular_total_multas_por_mes_com_nome(b):
        output_resultado.clear_output() 
        
        query = '''
        SELECT TO_CHAR(Penalidade.Data_de_aplicação, 'Month') AS Nome_Mes, SUM(Penalidade.Valor_multa) AS Total_Multas
        FROM Penalidade
        WHERE EXTRACT(YEAR FROM Penalidade.Data_de_aplicação) = 2023
        GROUP BY Nome_Mes
        ORDER BY EXTRACT(MONTH FROM MIN(Penalidade.Data_de_aplicação));
        '''
        
        df = pd.read_sql(query, conn)
        
        with output_resultado:
            display(df)

    # Função para obter os gêneros de livros mais populares (mais emprestados)
    def generos_mais_populares(b):
        output_resultado.clear_output() 
        
        query = '''
        SELECT Livro_Genero.Genero, COUNT(Realiza.ID_emprestimo) AS Quantidade_Emprestimos
        FROM Livro_Genero
        JOIN Realiza ON Livro_Genero.ISBN = Realiza.ISBN
        GROUP BY Livro_Genero.Genero
        ORDER BY Quantidade_Emprestimos DESC;
        '''
        
        df = pd.read_sql(query, conn)
        
        with output_resultado:
            display(df)

    # Função para calcular a média de dias de atraso nos empréstimos de livros escritos por autores britânicos
    def media_dias_atraso_autores_britanicos(b):
        output_resultado.clear_output()
        
        query = '''
        SELECT ROUND(AVG(Emprestimo.Dias_de_atraso)::NUMERIC, 2) AS Media_Dias_Atraso
        FROM Emprestimo
        JOIN Livro ON Emprestimo.ISBN = Livro.ISBN
        JOIN Escreve ON Livro.ISBN = Escreve.ISBN
        JOIN Autor ON Escreve.ID_autor = Autor.ID_autor
        WHERE Autor.Nacionalidade = 'Britânica';
        '''
        
        df = pd.read_sql(query, conn)
        
        with output_resultado:
            display(df)

    # Função para calcular a quantidade de exemplares usados de livros de ficção científica disponíveis em cada editora
    def quantidade_exemplares_usados_ficcao_cientifica(b):
        output_resultado.clear_output()
        
        query = '''
        SELECT Editora.Nome, COUNT(Exemplar.ID_exemplar) AS Quantidade_Exemplares_Usados
        FROM Editora
        JOIN Livro ON Editora.ID_editora = Livro.ID_editora
        JOIN Exemplar ON Livro.ISBN = Exemplar.ISBN
        JOIN Livro_Genero ON Livro.ISBN = Livro_Genero.ISBN
        WHERE Livro_Genero.Genero = 'Ficção Científica' AND Exemplar.Usado = 'Sim'
        GROUP BY Editora.Nome;
        '''
        
        df = pd.read_sql(query, conn)
        
        with output_resultado:
            display(df)

    # Função para calcular a média de multas aplicadas aos membros que pegaram livros de ficção científica nos últimos 6 meses
    def media_multas_ficcao_cientifica_ultimos_6_meses(b):
        output_resultado.clear_output()  # Limpa a saída para exibir o novo resultado
        
        query = '''
        SELECT ROUND(AVG(Penalidade.Valor_multa)::numeric, 2) AS Media_Multas
        FROM Penalidade
        JOIN Realiza ON Penalidade.ID_emprestimo = Realiza.ID_emprestimo
        JOIN Emprestimo ON Realiza.ID_emprestimo = Emprestimo.ID_emprestimo
        JOIN Livro_Genero ON Emprestimo.ISBN = Livro_Genero.ISBN
        WHERE Livro_Genero.Genero = 'Ficção Científica' 
        AND Emprestimo.Data_de_retirada >= NOW() - INTERVAL '6 months';
        '''
        
        df = pd.read_sql(query, conn)
        
        with output_resultado:
            display(df)

    # Função para consultar os membros que tiveram mais de empréstimo atrasados
    def consultar_atrasos_membros(b):
        output_resultado.clear_output()
        
        query = '''
        SELECT Membro.CPF, Pessoa.Nome AS Nome_Membro, COUNT(Emprestimo.Dias_de_atraso) AS Atrasos
        FROM Membro
        JOIN Pessoa ON Membro.ID_pessoa = Pessoa.ID_pessoa
        JOIN Realiza ON Membro.ID_pessoa = Realiza.ID_pessoa
        JOIN Emprestimo ON Realiza.ID_emprestimo = Emprestimo.ID_emprestimo
        WHERE Emprestimo.Dias_de_atraso > 0
        GROUP BY Membro.CPF, Pessoa.Nome
        ORDER BY Atrasos DESC;
        '''
        
        df = pd.read_sql(query, conn)
        
        with output_resultado:
            display(df)

    # Associe as funções aos botões
    botao_consulta_autor.on_click(consultar_autor_por_nacionalidade)
    botao_consulta_livro.on_click(consultar_livro_por_titulo)
    botao_consultar_membros.on_click(consultar_membros_por_cpf)
    botao_consulta_editora.on_click(consultar_editora)
    botao_consulta_pessoa.on_click(consultar_pessoa_por_nome)
    botao_consultar_penalidade.on_click(consultar_penalidade)
    botao_consultar_exemplares.on_click(consultar_livro_com_mais_exemplares)
    botao_consultar_membros_bloqueados.on_click(consultar_membros_bloqueados)
    botao_calcular_media_dias_atraso.on_click(calcular_media_dias_atraso)
    botao_calcular_total_multas_por_mes.on_click(calcular_total_multas_por_mes_com_nome)
    botao_generos_mais_populares.on_click(generos_mais_populares)
    botao_media_dias_atraso_autores_britanicos.on_click(media_dias_atraso_autores_britanicos)
    botao_quantidade_exemplares_usados_ficcao_cientifica.on_click(quantidade_exemplares_usados_ficcao_cientifica)
    botao_media_multas_ficcao_cientifica_ultimos_6_meses.on_click(media_multas_ficcao_cientifica_ultimos_6_meses)
    botao_membros_que_tiveram_mais_atraso.on_click(consultar_atrasos_membros)

    # Exiba os widgets para consulta por nacionalidade de autor e título de livro
    with output_resultado:
        display(widgets.HBox([label_consulta_autor, nacionalidade_autor_widget, botao_consulta_autor]))
        display(widgets.HBox([label_consulta_livro, titulo_livro_widget, botao_consulta_livro]))
        display(widgets.HBox([label_consulta_membro, cpf_membro_widget, botao_consultar_membros]))
        display(widgets.HBox([label_consulta_editora, nome_editora_consulta_widget, botao_consulta_editora]))
        display(widgets.HBox([label_consulta_pessoa, nome_pessoa_widget, botao_consulta_pessoa]))
        display(widgets.HBox([label_consulta_penalidade, botao_consultar_penalidade]))
        display(widgets.HBox([label_consulta_exemplares, botao_consultar_exemplares]))
        display(widgets.HBox([label_consulta_bloqueados, botao_consultar_membros_bloqueados]))
        display(widgets.HBox([label_calcular_media_dias_atraso, botao_calcular_media_dias_atraso]))
        display(widgets.HBox([label_calcular_total_multas_por_mes, botao_calcular_total_multas_por_mes]))
        display(widgets.HBox([label_generos_mais_populares, botao_generos_mais_populares]))
        display(widgets.HBox([label_media_dias_atraso_autores_britanicos, botao_media_dias_atraso_autores_britanicos]))
        display(widgets.HBox([label_quantidade_exemplares_usados_ficcao_cientifica, botao_quantidade_exemplares_usados_ficcao_cientifica]))
        display(widgets.HBox([label_media_multas_ficcao_cientifica_ultimos_6_meses, botao_media_multas_ficcao_cientifica_ultimos_6_meses]))
        display(widgets.HBox([label_membros_com_mais_atrasos, botao_membros_que_tiveram_mais_atraso]))

# Associe a função ao botão "Executar Consulta"
botao_executar_consulta = widgets.Button(description="Executar Consulta")
botao_executar_consulta.on_click(executar_consulta)


# Exiba os widgets
display(widgets.VBox([botao_inclusao, botao_exclusao, botao_executar_consulta, output_resultado]))


C:\Users\danie\AppData\Local\Temp\ipykernel_6860\2523337297.py:507: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


C:\Users\danie\AppData\Local\Temp\ipykernel_6860\2523337297.py:284: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_verificacao = pd.read_sql(query_verificacao, conn)


C:\Users\danie\AppData\Local\Temp\ipykernel_6860\2523337297.py:507: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [ ]:
3749576289456

4444476289454